In [1]:
%matplotlib inline

import os
import csv
from collections import Counter

import numpy as np
import matplotlib.pyplot as plt

import mne

# MNE associated & pre-processing code
from mne.preprocessing import ICA
from autoreject import AutoReject

In [2]:
def get_files(files):
    """Return specified file type from list of files.
    Note: scrappy implementation - returns only first found file that matches pattern. 
    """
    
    for file in files:
        if seq in file:
            return file

# Load Data

In [3]:
# Set data path
dat_path = '/Users/trahimi1/Downloads/MIND_2018/Hackathon/Subjects/'
# ToDo: general stuff to get a list of available subjects

In [4]:
# Set subject to load
sub_num = 'A00054039'
subj_path = os.path.join(dat_path, sub_num, 'EEG', 'raw', 'raw_format')

In [5]:
# Get list of subject files
subj_files = os.listdir(subj_path)

subj_files = [file for file in subj_files if ('A0' in file and file.endswith('.raw'))]

### Event code descriptions:

__Contrats Change__
- 3 :  5  = start trial
- 4 :  8  = Target ON left
- 5 :  9  = Target ON right
- 1 : 12  = button press left
- 2 : 13  = button press right

__Naturalistic Viewing Paradigm__
- 33 :  81  = start of video 1
- 1 :  101  = stop of video 1
- 34 :  82  = start of video 2
- 2 :  102  = stop of video 2
- 35 :  83  = start of video 3
- 3 :  103  = stop of video 3


__Resting Paradigm__
- 1 :  20 = eyes open start
- 2 :  30 = eyes closed start


__Sequence Learning Paradigm__

- *DOT ON:*
- 6 : 11
- 7 : 12
- 8 : 13
- 9 : 14
- 10 : 15
- 11 : 16
- 12 : 17
- 13 : 18

- *DOT OFF:*
- 15 : 21
- 16 : 22
- 17 : 23 
- 18 : 24
- 19 : 25
- 20 : 26
- 21 : 27
- 22 : 28


__Surround Suppression Paradigm__
- 1 :  4 = stimulus ON
- 2 :  8 = stimulus OFF
(There are also two condotions in this experiment which says there's a background or there is no background)


__Symbol Serach Paradigm__
- 2 :  20 = start of new page
- 1 :  14 = response for trial


In [6]:
# Contrats Change Detection Paradigm
contrast_event = {'12  ': 1, '13  ': 2, '5   ': 3, '8   ': 4, '9   ': 5}
nat_view_event = {'81  ':33, '101  ':1, '82  ':34, '102  ':2, '83  ':35, '103  ':3}
resting_event = {'20  ':1, '30  ':2}
seq_learn_event = {'11  ':6, '12  ':7, '13  ':8, '14  ':9, '15  ':10, '16  ':11, '17  ':12, '18  ':13, '21  ':15, '22  ':16, '23  ':17, '24  ':18, '25  ':19, '26  ':20, '27  ':21, '28  ':22}
surr_supp_event = {'4  ':1, '8  ':2}
symb_srch_event = {'14  ':1, '20  ':2}

In [7]:
# Get specific block
task = {}
for counter_file in range(len(subj_files)):
    cur_eeg_file = subj_files[counter_file]
    raw = mne.io.read_raw_egi(os.path.join(subj_path, cur_eeg_file), preload=True)
    if raw.event_id == contrast_event:
        event_mapping = contrast_event
        task['paradigm_' + str(counter_file)] = "contrast change"
    elif raw.event_id == nat_view_event:
        event_mapping = nat_view_event
        task['paradigm_' + str(counter_file)] = "naturalistic viewing"           
    elif raw.event_id == resting_event:
        event_mapping = resting_event
        task['paradigm_' + str(counter_file)] = "resting"           
    elif raw.event_id == seq_learn_event:
        event_mapping = seq_learn_event
        task['paradigm_' + str(counter_file)] = "sequence learning"
    elif raw.event_id == surr_supp_event:
        event_mapping = surr_supp_event
        task['paradigm_' + str(counter_file)] = "surround suppression"
    elif raw.event_id == symb_srch_event:
        event_mapping = symb_srch_event
        task['paradigm_' + str(counter_file)] = "symbol search"
    else:
        print("event mapping doesn't match any of events' id")

task['paradigm_' + str(counter_file)]

Reading EGI header from /Users/trahimi1/Downloads/MIND_2018/Hackathon/Subjects/A00054039/EEG/raw/raw_format/A00054039010.raw...
    Reading events ...
    Assembling measurement info ...
Did not find any event code with more than one event.
    Synthesizing trigger channel "STI 014" ...
    Excluding events {} ...
Reading 0 ... 156002  =      0.000 ...   312.004 secs...


<ipython-input-7-4dc6bf001d70>:5: RuntimeWarning: Did not find any event code with more than one event.
  raw = mne.io.read_raw_egi(os.path.join(subj_path, cur_eeg_file), preload=True)


event mapping doesn't match any of events' id
Reading EGI header from /Users/trahimi1/Downloads/MIND_2018/Hackathon/Subjects/A00054039/EEG/raw/raw_format/A00054039004.raw...
    Reading events ...
    Assembling measurement info ...
    Synthesizing trigger channel "STI 014" ...
    Excluding events {101 , 102 , 103 , 104 , 106 , 11  , 12  , 13  , 14  , 15  , 16  , 17  , 18  , 20  , 21  , 22  , 23  , 24  , 25  , 26  , 27  , 28  , 30  , 31  , 32  , 33  , 34  , 35  , 5   , 50  , 81  , 82  , 83  , 84  , 86  , 9   , 90  , 91  , 92  , 93  , 94  , 95  , 96  , 97  } ...
Reading 0 ... 123701  =      0.000 ...   247.402 secs...
event mapping doesn't match any of events' id
Reading EGI header from /Users/trahimi1/Downloads/MIND_2018/Hackathon/Subjects/A00054039/EEG/raw/raw_format/A00054039005.raw...
    Reading events ...
    Assembling measurement info ...
    Synthesizing trigger channel "STI 014" ...
    Excluding events {101 , 102 , 103 , 104 , 106 , 11  , 14  , 15  , 16  , 17  , 18  , 20  ,

<ipython-input-7-4dc6bf001d70>:5: RuntimeWarning: Did not find any event code with more than one event.
  raw = mne.io.read_raw_egi(os.path.join(subj_path, cur_eeg_file), preload=True)


event mapping doesn't match any of events' id
Reading EGI header from /Users/trahimi1/Downloads/MIND_2018/Hackathon/Subjects/A00054039/EEG/raw/raw_format/A00054039002.raw...
    Reading events ...
    Assembling measurement info ...
    Synthesizing trigger channel "STI 014" ...
    Excluding events {101 , 102 , 103 , 104 , 106 , 20  , 30  , 31  , 32  , 33  , 34  , 35  , 4   , 5   , 50  , 8   , 81  , 82  , 83  , 84  , 86  , 9   , 90  , 91  , 92  , 93  , 94  , 95  , 96  , 97  } ...
Reading 0 ... 117801  =      0.000 ...   235.602 secs...
event mapping doesn't match any of events' id
Reading EGI header from /Users/trahimi1/Downloads/MIND_2018/Hackathon/Subjects/A00054039/EEG/raw/raw_format/A00054039003.raw...
    Reading events ...
    Assembling measurement info ...
    Synthesizing trigger channel "STI 014" ...
    Excluding events {101 , 102 , 103 , 104 , 106 , 11  , 12  , 13  , 15  , 16  , 17  , 18  , 21  , 22  , 23  , 24  , 25  , 26  , 27  , 28  , 30  , 31  , 32  , 33  , 34  , 35  ,

<ipython-input-7-4dc6bf001d70>:5: RuntimeWarning: Did not find any event code with more than one event.
  raw = mne.io.read_raw_egi(os.path.join(subj_path, cur_eeg_file), preload=True)


event mapping doesn't match any of events' id
Reading EGI header from /Users/trahimi1/Downloads/MIND_2018/Hackathon/Subjects/A00054039/EEG/raw/raw_format/A00054039009.raw...
    Reading events ...
    Assembling measurement info ...
    Synthesizing trigger channel "STI 014" ...
    Excluding events {101 , 102 , 103 , 104 , 106 , 11  , 14  , 15  , 16  , 17  , 18  , 20  , 21  , 22  , 23  , 24  , 25  , 26  , 27  , 28  , 30  , 31  , 32  , 33  , 34  , 35  , 4   , 50  , 81  , 82  , 83  , 84  , 86  , 90  , 91  , 92  , 93  , 94  , 95  , 96  , 97  } ...
Reading 0 ... 127776  =      0.000 ...   255.552 secs...


'contrast change'

In [8]:
# data = {"Subject": sub_num}


# # download_dir = "data.csv" #where you want the file to be downloaded to 

# # csv = open(download_dir, "w") 
# # #"w" indicates that you're writing strings to the file


# for counter_file in range(len(subj_files)):
#     data.update({'str(file)': "task['paradigm_' + str(counter_file)]"})

# # csv.write("task['paradigm_' + str(counter_file)]")

# np.savetxt('data.csv', (sub_num, task), delimiter=',')
